In [1]:
import os 
import pandas as pd
import numpy as np

In [2]:


def chk_unique_eid(df):
    """
    Check unique eid number for a dataframe
    """
    print('loaded df has unique eid count: '+ str(len(df.eid.unique())))

    
    

def lst_ind(dfa_list,ind_val):
    """
    return a list of icd code that match with 'ind_val'
    """
    
    pre0=[]
    for i in dfa_list:
        if pd.isnull(i):
            pre0.append([])
        elif pd.notnull(i):
            si=[]
            jl=i.split(',')
            for ei in jl:
                ef=ei.replace(',','')
                efa,efb,efc=ef.partition(str(ind_val))
                if efa=='':
                    si.append(ef)
            pre0.append(si)
    return pre0

def hes_gen_ind_raw(icd,hesin_dfin,key_code,evnt, detail=False):
    """
    return a dataframe that contains indicator variable for a specific 'key_code' in HES data
        use 'detail= True' to get the detail matched code info
    """

    #dfc=load_data_by_fid(fid_int)
    #df_icd9m=dfc.copy()
    #dfa=hesin[['eid',str(icd)]].copy()
    dfa=hesin_dfin[['eid','record_id',str(icd)]].copy()


    
    dfa_lst=dfa[dfa.columns[dfa.columns.get_loc(str(icd))]].values.tolist()
    pre0=lst_ind(dfa_lst,str(key_code))

    gen_hes_name='hes_'+str(icd)+'_'+str(evnt)+str(key_code)
    gen_ind_name='ind_'+str(icd)+'_'+str(evnt)+str(key_code)

    dfa[str(gen_hes_name)]=pre0
    dfa[dfa.columns[dfa.columns.get_loc(str(gen_hes_name))]]=dfa[dfa.columns[dfa.columns.get_loc(str(gen_hes_name))]].apply(lambda y: np.nan if len(y)==0 else y )
    
    dfa[str(gen_ind_name)]=pre0
    dfa[dfa.columns[dfa.columns.get_loc(str(gen_ind_name))]]=dfa[dfa.columns[dfa.columns.get_loc(str(gen_ind_name))]].apply(lambda y: 0 if len(y)==0 else 1 )
    
    print('\nHES '+str(icd)+' ',str(evnt)+'('+str(key_code)+')'+' count: '+str(dfa[dfa.columns[dfa.columns.get_loc(str(gen_hes_name))]].count())+',\nFreq_tab \n'+str(dfa[dfa.columns[dfa.columns.get_loc(str(gen_ind_name))]].value_counts()))
    dfb=dfa[['eid','record_id',str(gen_ind_name)]]   
    
    if detail==True:
        return dfa
    else:
        return dfb




def hes_gen_ind_list(icd_in, hesin_dfin, key_code_list, evt, detai=False):
    """
    return a dataframe that contains indicator variables for each specific 'key_code' in 'key_code_list'
        use 'detai= True' to get the detail matched codes info
    """
    dfcl=[]
    if detai==False:
        for l in key_code_list:
            df_l=hes_gen_ind_raw(icd_in,hesin_dfin, l, str(evt), detail=False)
            dfcl.append(df_l)
        dfcl_merge=pd.concat(dfcl,axis=1)
        dfcl_merge=dfcl_merge.loc[:,~dfcl_merge.columns.duplicated()]  # drop duplicated 'eid' columns
        return dfcl_merge

    if detai==True:
        for l in key_code_list:
            df_l=hes_gen_ind_raw(icd_in,hesin_dfin, l, str(evt), detail=True)
            dfcl.append(df_l)
        dfcl_merge=pd.concat(dfcl,axis=1)
        dfcl_merge=dfcl_merge.loc[:,~dfcl_merge.columns.duplicated()]  # drop duplicated 'eid' columns
        return dfcl_merge


In [3]:
os.chdir("/oasis/scratch/comet/yhuan162/temp_project/HES")

#load ICD main diagnosis
hesin=pd.read_csv('ukb_hesin.tsv',delimiter='\t',encoding='utf-8')

#load ICD secondary diagnosis
hesin_icd9s=pd.read_csv('ukb_hesin_diag9_wide_vec.tsv',delimiter='\t',encoding='utf-8')
hesin_icd10s=pd.read_csv('ukb_hesin_diag10_wide_vec.tsv',delimiter='\t',encoding='utf-8')

In [4]:
hesin_icd10s.columns

Index(['eid', 'record_id', 'diag_icd10_sec', 'diag_icd10_nb_sec'], dtype='object')

In [5]:
%time hesin.head()
print('hesin records count: '+ str(hesin.eid.count()))
print('hesin unique eid count: '+ str(len(hesin.eid.unique())))

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 393 µs
hesin records count: 2577597
hesin unique eid count: 395859


In [6]:
%time hesin_icd9s.head()
print('hesin_icd9s records count: '+ str(hesin_icd9s.eid.count()))
print('hesin_icd9s unique eid count: '+ str(len(hesin_icd9s.eid.unique())))

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 346 µs
hesin_icd9s records count: 14808
hesin_icd9s unique eid count: 8716


In [7]:
%time hesin_icd10s.head()
print('hesin_icd10s records count: '+ str(hesin_icd10s.eid.count()))
print('hesin_icd10s unique eid count: '+ str(len(hesin_icd10s.eid.unique())))

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 287 µs
hesin_icd10s records count: 1694827
hesin_icd10s unique eid count: 320334


In [8]:
#hesin.columns.tolist()
#hesin_icd10s.columns=['eid','record_id','diag_icd10_sec','diag_icd10_nb_sec']
#hesin_icd9s.columns=['eid','record_id','diag_icd9_sec','diag_icd9_nb_sec']

In [9]:
### merge 9s / 10s to hesin
hesin_add_sec9=pd.merge(hesin, hesin_icd9s, on=['eid','record_id'], how='outer')
hesin_add_sec910=pd.merge(hesin_add_sec9, hesin_icd10s, on=['eid','record_id'], how='outer')
hesin_add_sec910.to_csv('hesin_add_sec910.tsv', index=None, sep='\t')

In [10]:
hesin_add_sec910.head()

,eid,record_id,admidate,anagest,anasdate,cause_icd10,cause_icd10_nb,diag_icd10,diag_icd10_nb,diag_icd9,...,numbaby,numpreg,opdate,oper4,oper4_nb,operstat,diag_icd9_sec,diag_icd9_nb_sec,diag_icd10_sec,diag_icd10_nb_sec
0,1772719,1071463,2003-05-15,NaN,NaN,NaN,NaN,R198,NaN,NaN,...,NaN,NaN,NaN,X998,NaN,NaN,NaN,NaN,Z530,NaN
1,1772719,1077874,2003-06-05,NaN,NaN,NaN,NaN,R104,NaN,NaN,...,NaN,NaN,2003-06-05,H251,NaN,NaN,NaN,NaN,K529,NaN
2,1772719,1127881,2000-05-01,NaN,NaN,NaN,NaN,M512,NaN,NaN,...,NaN,NaN,NaN,X998,NaN,NaN,NaN,NaN,NaN,NaN
3,1277767,3208109,NaN,NaN,NaN,NaN,NaN,M8414,NaN,NaN,...,NaN,NaN,2005-10-05,W201,NaN,NaN,NaN,NaN,NaN,NaN
4,1277767,4218015,NaN,NaN,NaN,NaN,NaN,M8414,NaN,NaN,...,NaN,NaN,2005-10-05,W201,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:

print('main diag count: '+str(hesin.eid.count()))
print('secondary icd 9 diag count: '+str(hesin_icd9s.eid.count()))
print('secondary icd 10 diag count: '+str(hesin_icd10s.eid.count()))
hesin_icd10s.head()
hesin_icd9s.head()
hesin_add_sec910.columns.tolist()

main diag count: 2577597
secondary icd 9 diag count: 14808
secondary icd 10 diag count: 1694827


['eid',
 'record_id',
 'admidate',
 'anagest',
 'anasdate',
 'cause_icd10',
 'cause_icd10_nb',
 'diag_icd10',
 'diag_icd10_nb',
 'diag_icd9',
 'diag_icd9_nb',
 'disdate',
 'epiend',
 'epistart',
 'matage',
 'neocare',
 'numbaby',
 'numpreg',
 'opdate',
 'oper4',
 'oper4_nb',
 'operstat',
 'diag_icd9_sec',
 'diag_icd9_nb_sec',
 'diag_icd10_sec',
 'diag_icd10_nb_sec']

In [12]:
#hesin_allicd=hesin_add_sec910[['eid', 'record_id', 'admidate', 'anasdate', 'diag_icd10', 'diag_icd9',
#                              'disdate', 'epiend', 'epistart', 'opdate', 'oper4', 'arr_index_icd9_sec',
#                              'diag_icd9_sec', 'arr_index_icd10_sec', 'diag_icd10_sec']]
hesin_allicd=hesin_add_sec910.copy()
hesin_allicd.columns

Index(['eid', 'record_id', 'admidate', 'anagest', 'anasdate', 'cause_icd10',
       'cause_icd10_nb', 'diag_icd10', 'diag_icd10_nb', 'diag_icd9',
       'diag_icd9_nb', 'disdate', 'epiend', 'epistart', 'matage', 'neocare',
       'numbaby', 'numpreg', 'opdate', 'oper4', 'oper4_nb', 'operstat',
       'diag_icd9_sec', 'diag_icd9_nb_sec', 'diag_icd10_sec',
       'diag_icd10_nb_sec'],
      dtype='object')

In [13]:
hes_icd10s_T1D=hes_gen_ind_list(icd_in='diag_icd10_sec',hesin_dfin=hesin_allicd, evt='T1D',key_code_list=['E10'])
hes_icd9s_T1D=hes_gen_ind_list(icd_in='diag_icd9_sec',hesin_dfin=hesin_allicd, evt='T1D',key_code_list=['25001','25011','25021','25091'])



HES diag_icd10_sec  T1D(E10) count: 13969,
Freq_tab 
0    2563628
1      13969
Name: ind_diag_icd10_sec_T1DE10, dtype: int64

HES diag_icd9_sec  T1D(25001) count: 14,
Freq_tab 
0    2577583
1         14
Name: ind_diag_icd9_sec_T1D25001, dtype: int64

HES diag_icd9_sec  T1D(25011) count: 1,
Freq_tab 
0    2577596
1          1
Name: ind_diag_icd9_sec_T1D25011, dtype: int64

HES diag_icd9_sec  T1D(25021) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_sec_T1D25021, dtype: int64

HES diag_icd9_sec  T1D(25091) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_sec_T1D25091, dtype: int64


In [14]:
%%time
hes_icd10m_T1D=hes_gen_ind_list(icd_in='diag_icd10',hesin_dfin=hesin_allicd, evt='T1D',key_code_list=['E10'])
hes_icd9m_T1D=hes_gen_ind_list(icd_in='diag_icd9',hesin_dfin=hesin_allicd, evt='T1D',key_code_list=['25001','25011','25021','25091'])

hes_icd10m_T2D=hes_gen_ind_list(icd_in='diag_icd10',hesin_dfin=hesin_allicd, evt='T2D',key_code_list=['E11'])
hes_icd9m_T2D=hes_gen_ind_list(icd_in='diag_icd9',hesin_dfin=hesin_allicd, evt='T2D',key_code_list=['25000','25020','25090'])

hes_icd10m_T2Dex=hes_gen_ind_list(icd_in='diag_icd10',hesin_dfin=hesin_allicd, evt='T2Dex',key_code_list=['E111','E116','E131'])
hes_icd9m_T2Dex=hes_gen_ind_list(icd_in='diag_icd9',hesin_dfin=hesin_allicd, evt='T2Dex',key_code_list=['25010'])


HES diag_icd10  T1D(E10) count: 2701,
Freq_tab 
0    2574896
1       2701
Name: ind_diag_icd10_T1DE10, dtype: int64

HES diag_icd9  T1D(25001) count: 19,
Freq_tab 
0    2577578
1         19
Name: ind_diag_icd9_T1D25001, dtype: int64

HES diag_icd9  T1D(25011) count: 2,
Freq_tab 
0    2577595
1          2
Name: ind_diag_icd9_T1D25011, dtype: int64

HES diag_icd9  T1D(25021) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_T1D25021, dtype: int64

HES diag_icd9  T1D(25091) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_T1D25091, dtype: int64

HES diag_icd10  T2D(E11) count: 3124,
Freq_tab 
0    2574473
1       3124
Name: ind_diag_icd10_T2DE11, dtype: int64

HES diag_icd9  T2D(25000) count: 39,
Freq_tab 
0    2577558
1         39
Name: ind_diag_icd9_T2D25000, dtype: int64

HES diag_icd9  T2D(25020) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_T2D25020, dtype: int64

HES diag_icd9  T2D(25090) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_T2D25090, dtype: int64

HES

In [15]:
%%time
hes_icd10s_T1D=hes_gen_ind_list(icd_in='diag_icd10_sec',hesin_dfin=hesin_allicd, evt='T1D',key_code_list=['E10'])
hes_icd9s_T1D=hes_gen_ind_list(icd_in='diag_icd9_sec',hesin_dfin=hesin_allicd, evt='T1D',key_code_list=['25001','25011','25021','25091'])

hes_icd10s_T2D=hes_gen_ind_list(icd_in='diag_icd10_sec',hesin_dfin=hesin_allicd, evt='T2D',key_code_list=['E11'])
hes_icd9s_T2D=hes_gen_ind_list(icd_in='diag_icd9_sec',hesin_dfin=hesin_allicd, evt='T2D',key_code_list=['25000','25020','25090'])

hes_icd10s_T2Dex=hes_gen_ind_list(icd_in='diag_icd10_sec',hesin_dfin=hesin_allicd, evt='T2Dex',key_code_list=['E111','E116','E131'])
hes_icd9s_T2Dex=hes_gen_ind_list(icd_in='diag_icd9_sec',hesin_dfin=hesin_allicd, evt='T2Dex',key_code_list=['25010'])


HES diag_icd10_sec  T1D(E10) count: 13969,
Freq_tab 
0    2563628
1      13969
Name: ind_diag_icd10_sec_T1DE10, dtype: int64

HES diag_icd9_sec  T1D(25001) count: 14,
Freq_tab 
0    2577583
1         14
Name: ind_diag_icd9_sec_T1D25001, dtype: int64

HES diag_icd9_sec  T1D(25011) count: 1,
Freq_tab 
0    2577596
1          1
Name: ind_diag_icd9_sec_T1D25011, dtype: int64

HES diag_icd9_sec  T1D(25021) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_sec_T1D25021, dtype: int64

HES diag_icd9_sec  T1D(25091) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_sec_T1D25091, dtype: int64

HES diag_icd10_sec  T2D(E11) count: 116784,
Freq_tab 
0    2460813
1     116784
Name: ind_diag_icd10_sec_T2DE11, dtype: int64

HES diag_icd9_sec  T2D(25000) count: 31,
Freq_tab 
0    2577566
1         31
Name: ind_diag_icd9_sec_T2D25000, dtype: int64

HES diag_icd9_sec  T2D(25020) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_sec_T2D25020, dtype: int64

HES diag_icd9_sec  T2D(25090) count: 0,

In [16]:
df_icd_T1D_list=[hes_icd10m_T1D, hes_icd10s_T1D, hes_icd9m_T1D, hes_icd9s_T1D]
df_icd_T2D_list=[hes_icd10m_T2D, hes_icd10s_T2D, hes_icd9m_T2D, hes_icd9s_T2D]
df_icd_T2Dex_list=[hes_icd10m_T2Dex, hes_icd10s_T2Dex, hes_icd9m_T2Dex, hes_icd9s_T2Dex]

In [17]:
icd_T1D_ind_pre=pd.concat(df_icd_T1D_list,axis=1)
icd_T1D_ind_pre=icd_T1D_ind_pre.loc[:,~icd_T1D_ind_pre.columns.duplicated()] 
icd_T1D_ind=icd_T1D_ind_pre[['eid','record_id']].copy()
icd_T1D_ind['icd_T1D_ind']=icd_T1D_ind_pre.drop(['eid','record_id'],axis=1).sum(axis=1)
icd_T1D_ind.icd_T1D_ind=icd_T1D_ind.icd_T1D_ind.apply(lambda y: 1 if y>0 else y)

icd_T2D_ind_pre=pd.concat(df_icd_T2D_list,axis=1)
icd_T2D_ind_pre=icd_T2D_ind_pre.loc[:,~icd_T2D_ind_pre.columns.duplicated()] 
icd_T2D_ind=icd_T2D_ind_pre[['eid','record_id']].copy()
icd_T2D_ind['icd_T2D_ind']=icd_T2D_ind_pre.drop(['eid','record_id'],axis=1).sum(axis=1)
icd_T2D_ind.icd_T2D_ind=icd_T2D_ind.icd_T2D_ind.apply(lambda y: 1 if y>0 else y)

icd_T2Dex_ind_pre=pd.concat(df_icd_T2Dex_list,axis=1)
icd_T2Dex_ind_pre=icd_T2Dex_ind_pre.loc[:,~icd_T2Dex_ind_pre.columns.duplicated()] 
icd_T2Dex_ind=icd_T2Dex_ind_pre[['eid','record_id']].copy()
icd_T2Dex_ind['icd_T2Dex_ind']=icd_T2Dex_ind_pre.drop(['eid','record_id'],axis=1).sum(axis=1)
icd_T2Dex_ind.icd_T2Dex_ind=icd_T2Dex_ind.icd_T2Dex_ind.apply(lambda y: 1 if y>0 else y)

In [18]:
print('T1D freq:')
print(icd_T1D_ind.icd_T1D_ind.value_counts())
print('\nT2D freq:')
print(icd_T2D_ind.icd_T2D_ind.value_counts())
print('\nT2Dex freq:')
print(icd_T2Dex_ind.icd_T2Dex_ind.value_counts())

T1D freq:
0    2561163
1      16434
Name: icd_T1D_ind, dtype: int64

T2D freq:
0    2457898
1     119699
Name: icd_T2D_ind, dtype: int64

T2Dex freq:
0    2576961
1        636
Name: icd_T2Dex_ind, dtype: int64


In [19]:
icd_ind_list=[icd_T1D_ind,icd_T2D_ind,icd_T2Dex_ind]
icd_ind=pd.concat(icd_ind_list,axis=1)
icd_ind=icd_ind.loc[:,~icd_ind.columns.duplicated()] 

In [20]:
len(icd_ind.eid.unique())
cond1=icd_ind.icd_T1D_ind==1
cond2=icd_ind.icd_T2D_ind==1
cond3=icd_ind.icd_T2Dex_ind==1

In [21]:
T2D_rec=icd_ind[~cond1 & cond2 & ~cond3]

In [22]:
len(T2D_rec.eid.unique())

24121

In [24]:
#list(T2D_rec.eid.unique())
icd_ind.head()

,eid,record_id,icd_T1D_ind,icd_T2D_ind,icd_T2Dex_ind
0,1772719,1071463,0,0,0
1,1772719,1077874,0,0,0
2,1772719,1127881,0,0,0
3,1277767,3208109,0,0,0
4,1277767,4218015,0,0,0


In [25]:
%%time
icd_ind['vec_icd_3ind']=icd_ind[icd_ind.columns[icd_ind.columns.get_loc('icd_T1D_ind'):]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)


CPU times: user 6min 44s, sys: 3.32 s, total: 6min 47s
Wall time: 6min 44s


In [26]:
icd_ind.head()

,eid,record_id,icd_T1D_ind,icd_T2D_ind,icd_T2Dex_ind,vec_icd_3ind
0,1772719,1071463,0,0,0,"0,0,0"
1,1772719,1077874,0,0,0,"0,0,0"
2,1772719,1127881,0,0,0,"0,0,0"
3,1277767,3208109,0,0,0,"0,0,0"
4,1277767,4218015,0,0,0,"0,0,0"


In [27]:
icd_ind['HES_T2D_ind']=icd_ind.vec_icd_3ind.apply(lambda y: 1 if y=='0,1,0' else 0)
icd_ind.HES_T2D_ind.value_counts()

0    2458580
1     119017
Name: HES_T2D_ind, dtype: int64

In [30]:
icd_ind.head()

,eid,record_id,icd_T1D_ind,icd_T2D_ind,icd_T2Dex_ind,vec_icd_3ind,HES_T2D_ind
0,1772719,1071463,0,0,0,"0,0,0",0
1,1772719,1077874,0,0,0,"0,0,0",0
2,1772719,1127881,0,0,0,"0,0,0",0
3,1277767,3208109,0,0,0,"0,0,0",0
4,1277767,4218015,0,0,0,"0,0,0",0


In [32]:
len(icd_ind[icd_ind.HES_T2D_ind==1].eid.unique())

24121

In [84]:
HES_T2D_case = icd_ind[icd_ind.HES_T2D_ind==1]
HES_T2D_case.to_csv('/temp_project/HES/outcomes/T2D/HES_T2D_case.tsv', index=False, sep='\t')

In [37]:
icd_ind.to_csv('/temp_project/HES/outcomes/T2D/HES_T2D_ind.tsv', index=False, sep='\t')

In [38]:
icd_ind=pd.read_csv('/temp_project/HES/outcomes/T2D/HES_T2D_ind.tsv', sep='\t')
icd_ind.columns.tolist()

['eid',
 'record_id',
 'icd_T1D_ind',
 'icd_T2D_ind',
 'icd_T2Dex_ind',
 'vec_icd_3ind',
 'HES_T2D_ind']

In [40]:
T2D_ind=icd_ind[['eid','record_id','HES_T2D_ind']]

In [41]:
hesin_T2D=pd.merge(hesin, T2D_ind, on=['eid','record_id'], how='outer')
hesin_T2D.to_csv('/temp_project/HES/outcomes/T2D/hesin_T2D.tsv', index=None, sep='\t')

In [42]:
hesin_T2D.HES_T2D_ind.value_counts()

0    2458580
1     119017
Name: HES_T2D_ind, dtype: int64

In [43]:
### Get date var from HES


hesin_date=hesin[['eid','record_id','admidate','anasdate','disdate','epiend','epistart','opdate']].copy()
hesin_date['admidate'] = pd.to_datetime(hesin_date['admidate'])
hesin_date['anasdate'] = pd.to_datetime(hesin_date['anasdate'])
hesin_date['disdate'] = pd.to_datetime(hesin_date['disdate'])
hesin_date['epiend'] = pd.to_datetime(hesin_date['epiend'])
hesin_date['epistart'] = pd.to_datetime(hesin_date['epistart'])
hesin_date['opdate'] = pd.to_datetime(hesin_date['opdate'])

hesin_date['any_date']=hesin_date.drop(['eid','record_id'],axis=1).min(axis=1)
hesin_date['any_yr']=hesin_date['any_date'].dt.year
print('record count: '+str(hesin_date.eid.count()))
print('any_date count: '+str(hesin_date.any_date.count()))

hesin_date.to_csv('hesin_date.tsv', sep='\t', index=None)




### load date from ukbb
ukbb_date = pd.read_csv('/temp_project/ukbb/data/dfoi/ukbb_date.csv')
ukbb_date['att_date'] = pd.to_datetime(ukbb_date['att_date'])


hesin_ukbb_date = pd.merge(hesin_date, ukbb_date, on='eid', how='left')

record count: 2577597
any_date count: 2577595


In [44]:
ukbb_date.head()

,eid,att_date,att_yr
0,1000011,2008-01-07,2008
1,1000026,2009-03-04,2009
2,1000032,2008-04-11,2008
3,1000044,2009-01-14,2009
4,1000058,2009-02-25,2009


In [45]:
hesin_date.head()

,eid,record_id,admidate,anasdate,disdate,epiend,epistart,opdate,any_date,any_yr
0,1772719,1071463,2003-05-15,NaT,2003-05-15,2003-05-15,2003-05-15,NaT,2003-05-15,2003.0
1,1772719,1077874,2003-06-05,NaT,2003-06-05,2003-06-05,2003-06-05,2003-06-05,2003-06-05,2003.0
2,1772719,1127881,2000-05-01,NaT,2000-05-01,2000-05-01,2000-05-01,NaT,2000-05-01,2000.0
3,1277767,3208109,NaT,NaT,NaT,2005-10-05,2005-10-05,2005-10-05,2005-10-05,2005.0
4,1277767,4218015,NaT,NaT,NaT,2005-10-05,2005-10-05,2005-10-05,2005-10-05,2005.0


In [46]:
hesin_date.to_csv('hesin_date.tsv', sep='\t', index=None)

In [47]:
hesin_ukbb_date = pd.merge(hesin_date, ukbb_date, on='eid', how='left')

In [48]:
hesin_date.head()

,eid,record_id,admidate,anasdate,disdate,epiend,epistart,opdate,any_date,any_yr
0,1772719,1071463,2003-05-15,NaT,2003-05-15,2003-05-15,2003-05-15,NaT,2003-05-15,2003.0
1,1772719,1077874,2003-06-05,NaT,2003-06-05,2003-06-05,2003-06-05,2003-06-05,2003-06-05,2003.0
2,1772719,1127881,2000-05-01,NaT,2000-05-01,2000-05-01,2000-05-01,NaT,2000-05-01,2000.0
3,1277767,3208109,NaT,NaT,NaT,2005-10-05,2005-10-05,2005-10-05,2005-10-05,2005.0
4,1277767,4218015,NaT,NaT,NaT,2005-10-05,2005-10-05,2005-10-05,2005-10-05,2005.0


In [49]:
hesin_T2D_merge=hesin_T2D.drop(['admidate','anasdate','disdate','epiend','epistart','opdate'],axis=1)

In [50]:
hesin_T2D_merge.columns

Index(['eid', 'record_id', 'anagest', 'cause_icd10', 'cause_icd10_nb',
       'diag_icd10', 'diag_icd10_nb', 'diag_icd9', 'diag_icd9_nb', 'matage',
       'neocare', 'numbaby', 'numpreg', 'oper4', 'oper4_nb', 'operstat',
       'HES_T2D_ind'],
      dtype='object')

In [51]:
hesin_T2D_wd=pd.merge(hesin_T2D_merge, hesin_date, on=['eid','record_id'], how='outer')

In [52]:
hesin_T2D_wd.count()

eid               2577597
record_id         2577597
anagest             10263
cause_icd10        101660
cause_icd10_nb        208
diag_icd10        2512419
diag_icd10_nb        5244
diag_icd9           52121
diag_icd9_nb            0
matage              14321
neocare           1848633
numbaby             31070
numpreg             31063
oper4             2386150
oper4_nb                0
operstat          1372220
HES_T2D_ind       2577597
admidate          2287724
anasdate            13121
disdate           2172998
epiend            2518087
epistart          2520356
opdate            1914276
any_date          2577595
any_yr            2577595
dtype: int64

In [53]:
hesin_T2D_wd[hesin_T2D_wd.any_date.isna()].to_csv('any_date_miss.csv', index=None,)

In [55]:
pwd()

'/oasis/scratch/comet/yhuan162/temp_project/HES'

In [54]:
hesin_T2D_wd.to_pickle("/temp_project/HES/outcomes/T2D/hesin_T2D_wd.pkl")

In [44]:
hesin_T2D_wd=pd.read_pickle("/temp_project/HES/outcomes/T2D/hesin_T2D_wd.pkl")

In [56]:
hesin_T2D_wd.head()

,eid,record_id,anagest,cause_icd10,cause_icd10_nb,diag_icd10,diag_icd10_nb,diag_icd9,diag_icd9_nb,matage,...,operstat,HES_T2D_ind,admidate,anasdate,disdate,epiend,epistart,opdate,any_date,any_yr
0,1772719,1071463,NaN,NaN,NaN,R198,NaN,NaN,NaN,NaN,...,NaN,0,2003-05-15,NaT,2003-05-15,2003-05-15,2003-05-15,NaT,2003-05-15,2003.0
1,1772719,1077874,NaN,NaN,NaN,R104,NaN,NaN,NaN,NaN,...,NaN,0,2003-06-05,NaT,2003-06-05,2003-06-05,2003-06-05,2003-06-05,2003-06-05,2003.0
2,1772719,1127881,NaN,NaN,NaN,M512,NaN,NaN,NaN,NaN,...,NaN,0,2000-05-01,NaT,2000-05-01,2000-05-01,2000-05-01,NaT,2000-05-01,2000.0
3,1277767,3208109,NaN,NaN,NaN,M8414,NaN,NaN,NaN,NaN,...,NaN,0,NaT,NaT,NaT,2005-10-05,2005-10-05,2005-10-05,2005-10-05,2005.0
4,1277767,4218015,NaN,NaN,NaN,M8414,NaN,NaN,NaN,NaN,...,NaN,0,NaT,NaT,NaT,2005-10-05,2005-10-05,2005-10-05,2005-10-05,2005.0


In [57]:
hesin_T2D_wd.columns.tolist()

['eid',
 'record_id',
 'anagest',
 'cause_icd10',
 'cause_icd10_nb',
 'diag_icd10',
 'diag_icd10_nb',
 'diag_icd9',
 'diag_icd9_nb',
 'matage',
 'neocare',
 'numbaby',
 'numpreg',
 'oper4',
 'oper4_nb',
 'operstat',
 'HES_T2D_ind',
 'admidate',
 'anasdate',
 'disdate',
 'epiend',
 'epistart',
 'opdate',
 'any_date',
 'any_yr']

In [58]:
hesin_T2D_wd_wrk=hesin_T2D_wd.copy()

In [59]:
#tcond1=hesin_T2D_wd_wrk['any_date']<pd.to_datetime("2006-01-01")
#tcond2=     (hesin_T2D_wd_wrk['any_date'] >= pd.to_datetime("2006-01-01")) & (hesin_T2D_wd_wrk['any_date'] < pd.to_datetime("2011-01-01")),
#tcond3= hesin_T2D_wd_wrk['any_date']>=pd.to_datetime("2011-01-01")

In [60]:
conditions = [
    (hesin_T2D_wd_wrk['any_date'] < pd.to_datetime("2006-01-01")),
    (hesin_T2D_wd_wrk['any_date'] >= pd.to_datetime("2006-01-01")) & (hesin_T2D_wd_wrk['any_date'] < pd.to_datetime("2011-01-01")),
    (hesin_T2D_wd_wrk['any_date'] >= pd.to_datetime("2011-01-01"))]
choices = [1, 2, 3]
hesin_T2D_wd_wrk['time_in'] = np.select(conditions, choices, default=0)

In [61]:
hesin_T2D_wd_wrk.time_in.value_counts()

3    916725
1    895436
2    765434
0         2
Name: time_in, dtype: int64

In [62]:
hesin_T2D_wd_wrk_ukbb_date = pd.merge(hesin_T2D_wd_wrk, ukbb_date, on='eid', how='left')

In [63]:
### drop na
hesin_T2D_wd_wrk_ukbb_date2=hesin_T2D_wd_wrk_ukbb_date[['eid','record_id','HES_T2D_ind','any_date','any_yr','time_in','att_date', 'att_yr']].dropna().copy()
### extract year
#hesin_T2D_wd_wrk_ukbb_date2['any_yr']=hesin_T2D_wd_wrk_ukbb_date2['any_date'].dt.year

In [64]:
hesin_T2D_wd_wrk_ukbb_date2['diff_yr']=hesin_T2D_wd_wrk_ukbb_date2['any_yr']-hesin_T2D_wd_wrk_ukbb_date2['att_yr']
hesin_T2D_wd_wrk_ukbb_date2['diff_yr_cat']=hesin_T2D_wd_wrk_ukbb_date2['diff_yr'].apply(lambda y: 1 if y<0 else (2 if y==0 else (3 if str(y)!='nan' else np.nan)))
# diff_yr_cat = 1, any_yr < att_yr: pre baseline
# diff_yr_cat = 2, any_yr = att_yr: at baseline
# diff_yr_cat = 3, any_yr > att_yr: post baseline

In [65]:
hesin_T2D_wd_wrk_ukbb_date2.head()

,eid,record_id,HES_T2D_ind,any_date,any_yr,time_in,att_date,att_yr,diff_yr,diff_yr_cat
0,1772719,1071463,0,2003-05-15,2003.0,1,2009-09-16,2009,-6.0,1
1,1772719,1077874,0,2003-06-05,2003.0,1,2009-09-16,2009,-6.0,1
2,1772719,1127881,0,2000-05-01,2000.0,1,2009-09-16,2009,-9.0,1
3,1277767,3208109,0,2005-10-05,2005.0,1,2008-02-07,2008,-3.0,1
4,1277767,4218015,0,2005-10-05,2005.0,1,2008-02-07,2008,-3.0,1


In [66]:
len(hesin_T2D_wd_wrk_ukbb_date2[hesin_T2D_wd_wrk_ukbb_date2.diff_yr<=2].eid.unique())

354027

In [67]:
len(hesin_T2D_wd_wrk_ukbb_date2.eid.unique())   #395859
len(hesin_T2D_wd_wrk_ukbb_date2[hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat==1].eid.unique())  #309656
len(hesin_T2D_wd_wrk_ukbb_date2[hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat==2].eid.unique())  #74922
len(hesin_T2D_wd_wrk_ukbb_date2[hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat==3].eid.unique())  #271993
len(hesin_T2D_wd_wrk_ukbb_date2[hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat!=1].eid.unique())  #290294



len(hesin_T2D_wd_wrk_ukbb_date2[(hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat==1) 
                                & (hesin_T2D_wd_wrk_ukbb_date2.HES_T2D_ind==1)].eid.unique())  #8695
len(hesin_T2D_wd_wrk_ukbb_date2[(hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat==2) 
                                & (hesin_T2D_wd_wrk_ukbb_date2.HES_T2D_ind==1)].eid.unique())  #3695
len(hesin_T2D_wd_wrk_ukbb_date2[(hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat==3) 
                                & (hesin_T2D_wd_wrk_ukbb_date2.HES_T2D_ind==1)].eid.unique())  #20765
len(hesin_T2D_wd_wrk_ukbb_date2[(hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat!=1) 
                                & (hesin_T2D_wd_wrk_ukbb_date2.HES_T2D_ind==1)].eid.unique())  #21630


21630

In [68]:
df_pre_temp= hesin_T2D_wd_wrk_ukbb_date2[hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat==1]
df_at_temp = hesin_T2D_wd_wrk_ukbb_date2[hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat==2]
df_post_temp = hesin_T2D_wd_wrk_ukbb_date2[hesin_T2D_wd_wrk_ukbb_date2.diff_yr_cat==3]

In [69]:
df_pre_temp2=df_pre_temp.drop(['record_id','time_in','any_date','att_date'], axis=1).drop_duplicates().sort_values(by=['eid','any_yr','HES_T2D_ind'], ascending=[True, True, False])
df_pre_temp2['rec_pre']=df_pre_temp2.groupby(['eid'],sort=False).cumcount()+1
#df_pre_temp2[df_pre_temp2.eid==1001564]
df_pre_temp3=df_pre_temp2.set_index(['eid','rec_pre'])
df_pre_temp4=df_pre_temp3.unstack(level=1)
df_pre_temp4.columns = df_pre_temp4.columns.map('{0[0]}_pre{0[1]}'.format)
df_pre_temp5=df_pre_temp4.reset_index()
df_pre_temp5['vec_HES_T2D_ind_pre'] = df_pre_temp5[df_pre_temp5.columns[df_pre_temp5.columns.get_loc('HES_T2D_ind_pre1'):df_pre_temp5.columns.get_loc('any_yr_pre1')]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
df_pre_temp5['HES_T2D_ind_pre']=df_pre_temp5['vec_HES_T2D_ind_pre'].apply(lambda y: 1 if '1.0' in y.split(',') else 0)


In [70]:
df_at_temp2=df_at_temp.drop(['record_id','time_in','any_date','att_date'], axis=1).drop_duplicates().sort_values(by=['eid','any_yr','HES_T2D_ind'], ascending=[True, True, False])
df_at_temp2['rec_at']=df_at_temp2.groupby(['eid'],sort=False).cumcount()+1
df_at_temp3=df_at_temp2.set_index(['eid','rec_at'])
df_at_temp4=df_at_temp3.unstack(level=1)
df_at_temp4.columns = df_at_temp4.columns.map('{0[0]}_at{0[1]}'.format)
df_at_temp5=df_at_temp4.reset_index()
df_at_temp5['vec_HES_T2D_ind_at'] = df_at_temp5[df_at_temp5.columns[df_at_temp5.columns.get_loc('HES_T2D_ind_at1'):df_at_temp5.columns.get_loc('any_yr_at1')]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
df_at_temp5['HES_T2D_ind_at']=df_at_temp5['vec_HES_T2D_ind_at'].apply(lambda y: 1 if '1.0' in y.split(',') else 0)


df_post_temp2=df_post_temp.drop(['record_id','time_in','any_date','att_date'], axis=1).drop_duplicates().sort_values(by=['eid','any_yr','HES_T2D_ind'], ascending=[True, True, False])
df_post_temp2['rec_post']=df_post_temp2.groupby(['eid'],sort=False).cumcount()+1
df_post_temp3=df_post_temp2.set_index(['eid','rec_post'])
df_post_temp4=df_post_temp3.unstack(level=1)
df_post_temp4.columns = df_post_temp4.columns.map('{0[0]}_post{0[1]}'.format)
df_post_temp5=df_post_temp4.reset_index()
df_post_temp5['vec_HES_T2D_ind_post'] = df_post_temp5[df_post_temp5.columns[df_post_temp5.columns.get_loc('HES_T2D_ind_post1'):df_post_temp5.columns.get_loc('any_yr_post1')]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
df_post_temp5['HES_T2D_ind_post']=df_post_temp5['vec_HES_T2D_ind_post'].apply(lambda y: 1 if '1.0' in y.split(',') else 0)


In [71]:
df_pre_temp5.eid.count()
df_at_temp5.eid.count()
df_post_temp5.eid.count()

271993

In [72]:
df_pa_temp=pd.merge(df_pre_temp5, df_at_temp5, on='eid', how='outer')
df_pa_temp.eid.count() #325138
df_pap_temp=pd.merge(df_pa_temp, df_post_temp5, on='eid', how='outer')
df_pap_temp.eid.count() #395859

cf=df_pap_temp[['HES_T2D_ind_pre','HES_T2D_ind_at','HES_T2D_ind_post']].fillna(0)
tab3w  = pd.crosstab(index=[cf["HES_T2D_ind_pre"],cf["HES_T2D_ind_at"]], 
                             columns=[
                                      cf["HES_T2D_ind_post"]],
                             margins=True)  
tab3w

HES_T2D_ind_post                   0.0    1.0     All
HES_T2D_ind_pre HES_T2D_ind_at                       
0.0             0.0             371738  13738  385476
                1.0                531   1157    1688
1.0             0.0               2491   4197    6688
                1.0                334   1673    2007
All                             375094  20765  395859

In [73]:
papcond1=df_pap_temp.HES_T2D_ind_pre==1
papcond2=df_pap_temp.HES_T2D_ind_at==1
papcond3=df_pap_temp.HES_T2D_ind_post==1

In [103]:
df_p1a01p01 =df_pap_temp[papcond1 ]
df_p0a0p1   =df_pap_temp[~papcond1 & ~papcond2 & papcond3]
df_p0a1p01  =df_pap_temp[~papcond1 & papcond2 ]




df_p1a01p01.eid.count() #8695
df_p0a1p01.eid.count() #1688
df_p0a0p1.eid.count() #13738

df_p1a01p01_eid_list = df_p1a01p01.eid.tolist()
HES_T2D_case_eid_list = HES_T2D_case.eid.unique().tolist()
HES_T2D_case_eid_list

HES_T2D_case_pap = df_pap_temp[df_pap_temp.eid.isin(HES_T2D_case_eid_list)]

df_p0p01p01 = HES_T2D_case_pap[~HES_T2D_case_pap.eid.isin(df_p1a01p01_eid_list)]

In [105]:
HES_T2D_case_pap.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/HES_T2D_case_pap.csv',index=None)
df_p0p01p01.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_p0p01p01.csv',index=None)
df_p1a01p01.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_p1a01p01.csv',index=None)
df_p0a1p01.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_p0a1p01.csv',index=None)
df_p0a0p1.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_p0a0p1.csv',index=None)


In [168]:
#load ukbb T2D p1 and p2 indicators

T2D_case_p1=pd.read_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/T2D_case_p1.csv')
print('ukbb T2D case p1 count: ' + str(T2D_case_p1.eid.count()))

T2D_case_p2=pd.read_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/T2D_case_p2.csv')
print('ukbb T2D case p2 count: ' + str(T2D_case_p2.eid.count()))

T2D_case_p1p2_list=[T2D_case_p1, T2D_case_p2]
T2D_case_p1p2=pd.concat(T2D_case_p1p2_list,axis=0).copy()
print('ukbb T2D case p1 and p2 count: '+str(T2D_case_p1p2.eid.count()))


ukbb T2D case p1 count: 9569
ukbb T2D case p2 count: 598
ukbb T2D case p1 and p2 count: 10167


In [347]:
### ukbb1 and p0a0p1
df_ukbb1_p0a0p1=pd.merge(T2D_case_p1,df_p0a0p1,on='eid',how='inner')
df_ukbb1_p0a0p1.eid.count()   #4036
df_ukbb1_p0a0p1_eid_list=df_ukbb1_p0a0p1.eid.tolist()

df_ukbb0_p0a0p1=df_p0a0p1[~df_p0a0p1.eid.isin(df_ukbb1_p0a0p1_eid_list)]  #9702

df_ukbb1_p0a0p1.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_ukbb1_p0a0p1.csv', index=None)
df_ukbb0_p0a0p1.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_ukbb0_p0a0p1.csv', index=None)



### ukbb1 and p1a01p01
df_ukbb1_p1a01p01=pd.merge(T2D_case_p1,df_p1a01p01,on='eid',how='inner')
df_ukbb1_p1a01p01.eid.count()   #4677
df_ukbb1_p1a01p01_eid_list=df_ukbb1_p1a01p01.eid.tolist()

df_ukbb0_p1a01p01=df_p1a01p01[~df_p1a01p01.eid.isin(df_ukbb1_p1a01p01_eid_list)]   #4018

df_ukbb1_p1a01p01.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_ukbb1_p1a01p01.csv', index=None)
df_ukbb0_p1a01p01.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_ukbb0_p1a01p01.csv', index=None)



### ukbb1 and p0a1p01
df_ukbb1_p0a1p01=pd.merge(T2D_case_p1,df_p0a1p01,on='eid',how='inner')
df_ukbb1_p0a1p01.eid.count()   #855
df_ukbb1_p0a1p01_eid_list=df_ukbb1_p0a1p01.eid.tolist()

df_ukbb0_p0a1p01=df_p0a1p01[~df_p0a1p01.eid.isin(df_ukbb1_p0a1p01_eid_list)]  #833

df_ukbb1_p0a1p01.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_ukbb1_p0a1p01.csv', index=None)
df_ukbb0_p0a1p01.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_ukbb0_p0a1p01.csv', index=None)



In [192]:
### T2D diseased dataframe
df_T2D_diseased_list=[df_ukbb1_p0a0p1, df_ukbb1_p1a01p01, df_ukbb1_p0a1p01]
df_T2D_diseased=pd.concat(df_T2D_diseased_list, axis=0)




## save T2D diseased dataframe

In [346]:
df_T2D_diseased.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_T2D_diseased.csv', index=None)

In [220]:
### T2D incubation dataframe
df_T2D_incubation=df_ukbb0_p0a0p1.copy()


### build 2yr/5yr/5yr+ time window for incubating T2D dataframe
### create incubating indicator
df_T2D_incubation_eid=df_T2D_incubation[['eid']].copy()
df_T2D_incubation_eid['incu_ind']=np.repeat(1, df_T2D_incubation_eid.eid.count())


### select dataframe for generating time window
df_T2D_HES_ukbb_incu_temp=pd.merge(hesin_T2D_wd_wrk_ukbb_date2,df_T2D_incubation_eid, on='eid', how='left')
df_T2D_HES_ukbb_incu_temp2=df_T2D_HES_ukbb_incu_temp[df_T2D_HES_ukbb_incu_temp.incu_ind==1].copy()
len(df_T2D_HES_ukbb_incu_temp2.eid.unique())


9702

In [263]:
### 2yr window

df_T2D_HES_ukbb_incu_temp2['w2yr']=df_T2D_HES_ukbb_incu_temp2['diff_yr'].apply(lambda y: 0 if y<0 else (1 if y<=2 else 0))
df_T2D_HES_ukbb_incu_temp3=df_T2D_HES_ukbb_incu_temp2[df_T2D_HES_ukbb_incu_temp2.w2yr==1].copy()
#df_T2D_HES_ukbb_incu_w2yr_eid_list=df_T2D_HES_ukbb_incu_temp3.eid.unique.tolist()
len(df_T2D_HES_ukbb_incu_temp3.eid.unique())  #6096

df_T2D_HES_ukbb_incu_temp4=df_T2D_HES_ukbb_incu_temp3[['eid','HES_T2D_ind','w2yr']].drop_duplicates().sort_values(by=['eid','HES_T2D_ind'], ascending=[True, False])
len(df_T2D_HES_ukbb_incu_temp4.eid.unique())  #6096

### multiple T2D at same eid, only select one with T2D=1, drop others
df_T2D_HES_ukbb_incu_temp4.loc[ df_T2D_HES_ukbb_incu_temp4.groupby(['eid'],as_index=False).nth([0]).index, 'flag_1st' ] = 1
df_T2D_HES_ukbb_incu_temp5 = df_T2D_HES_ukbb_incu_temp4.dropna()

df_T2D_HES_ukbb_incu_w2yr=df_T2D_HES_ukbb_incu_temp5[df_T2D_HES_ukbb_incu_temp5.HES_T2D_ind==1].copy() #2765
df_T2D_HES_ukbb_incu_w2yr_eid_list=df_T2D_HES_ukbb_incu_temp5[df_T2D_HES_ukbb_incu_temp5.HES_T2D_ind==1].eid.tolist()

df_T2D_HES_ukbb_incu_temp5.HES_T2D_ind.value_counts()
df_T2D_HES_ukbb_incu_w2yr.HES_T2D_ind.value_counts()
len(df_T2D_HES_ukbb_incu_w2yr.eid.unique())



### 5yr window

df_T2D_HES_ukbb_incu_w5yr_temp=df_T2D_HES_ukbb_incu_temp2[~df_T2D_HES_ukbb_incu_temp2.eid.isin(df_T2D_HES_ukbb_incu_w2yr_eid_list)].copy()
df_T2D_HES_ukbb_incu_w5yr_temp['w5yr']=df_T2D_HES_ukbb_incu_w5yr_temp['diff_yr'].apply(lambda y: 0 if y<=2 else (1 if y<=5 else 0))
len(df_T2D_HES_ukbb_incu_w5yr_temp.eid.unique())  #6937

df_T2D_HES_ukbb_incu_w5yr_temp2 = df_T2D_HES_ukbb_incu_w5yr_temp[df_T2D_HES_ukbb_incu_w5yr_temp.w5yr==1].copy()
len(df_T2D_HES_ukbb_incu_w5yr_temp2.eid.unique())  #5984
df_T2D_HES_ukbb_incu_w5yr_temp3 = df_T2D_HES_ukbb_incu_w5yr_temp2[['eid','HES_T2D_ind','w5yr']].drop_duplicates().sort_values(by=['eid','HES_T2D_ind'], ascending=[True, False])

### multiple T2D at same eid, only select one with T2D=1, drop others
df_T2D_HES_ukbb_incu_w5yr_temp3.loc[ df_T2D_HES_ukbb_incu_w5yr_temp3.groupby(['eid'],as_index=False).nth([0]).index, 'flag_1st' ] = 1
df_T2D_HES_ukbb_incu_w5yr_temp4 = df_T2D_HES_ukbb_incu_w5yr_temp3.dropna()

df_T2D_HES_ukbb_incu_w5yr=df_T2D_HES_ukbb_incu_w5yr_temp4[df_T2D_HES_ukbb_incu_w5yr_temp4.HES_T2D_ind==1].copy() #2765
df_T2D_HES_ukbb_incu_w5yr_eid_list=df_T2D_HES_ukbb_incu_w5yr_temp4[df_T2D_HES_ukbb_incu_w5yr_temp4.HES_T2D_ind==1].eid.tolist()

### 5yr+ window

df_T2D_HES_ukbb_incu_w5yrplus_temp=df_T2D_HES_ukbb_incu_temp2[(~df_T2D_HES_ukbb_incu_temp2.eid.isin(df_T2D_HES_ukbb_incu_w2yr_eid_list)) &
                          (~df_T2D_HES_ukbb_incu_temp2.eid.isin(df_T2D_HES_ukbb_incu_w5yr_eid_list))].copy()

len(df_T2D_HES_ukbb_incu_w5yrplus_temp.eid.unique())    #1812

df_T2D_HES_ukbb_incu_w5yrplus_temp2 = df_T2D_HES_ukbb_incu_w5yrplus_temp[df_T2D_HES_ukbb_incu_w5yrplus_temp.HES_T2D_ind==1].copy()
df_T2D_HES_ukbb_incu_w5yrplus_temp2['w5yrplus']=df_T2D_HES_ukbb_incu_w5yrplus_temp2['diff_yr'].apply(lambda y: 1 if y>5 else 0)
df_T2D_HES_ukbb_incu_w5yrplus = df_T2D_HES_ukbb_incu_w5yrplus_temp2[['eid', 'HES_T2D_ind', 'w5yrplus']].drop_duplicates().copy()

### merge 3 time windows

df_T2D_HES_ukbb_incu_w2yr_merge = df_T2D_HES_ukbb_incu_w2yr[['eid']].copy()
df_T2D_HES_ukbb_incu_w2yr_merge['w2yr_T2D'] = np.repeat(1, df_T2D_HES_ukbb_incu_w2yr_merge.eid.count())

df_T2D_HES_ukbb_incu_w5yr_merge = df_T2D_HES_ukbb_incu_w5yr[['eid']].copy()
df_T2D_HES_ukbb_incu_w5yr_merge['w5yr_T2D'] = np.repeat(1, df_T2D_HES_ukbb_incu_w5yr_merge.eid.count())

df_T2D_HES_ukbb_incu_w5yrplus_merge = df_T2D_HES_ukbb_incu_w5yrplus[['eid']].copy()
df_T2D_HES_ukbb_incu_w5yrplus_merge['w5yrplus_T2D'] = np.repeat(1, df_T2D_HES_ukbb_incu_w5yrplus_merge.eid.count())


df_T2D_HES_ukbb_incu_win1 = pd.merge(df_T2D_HES_ukbb_incu_w2yr_merge, df_T2D_HES_ukbb_incu_w5yr_merge, on='eid', how='outer')
df_T2D_HES_ukbb_incu_win2 = pd.merge(df_T2D_HES_ukbb_incu_win1, df_T2D_HES_ukbb_incu_w5yrplus_merge, on='eid', how='outer')

df_T2D_HES_ukbb_incu_with_time_window = df_T2D_HES_ukbb_incu_win2.fillna(0)
df_T2D_HES_ukbb_incu_with_time_window.count()


2765

## save T2D incubation dataframe

In [348]:
df_T2D_HES_ukbb_incu_with_time_window.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/df_T2D_incubation.csv', index=None)
df_T2D_HES_ukbb_incu_with_time_window.head()

,eid,w2yr_T2D,w5yr_T2D,w5yrplus_T2D
0,1002191,1.0,0.0,0.0
1,1005134,1.0,0.0,0.0
2,1011281,1.0,0.0,0.0
3,1015397,1.0,0.0,0.0
4,1016455,1.0,0.0,0.0


In [349]:
df_T2D_HES_ukbb_incu_win2.count()

eid             9702
w2yr_T2D        2765
w5yr_T2D        5125
w5yrplus_T2D    1812
dtype: int64

### Program ends here, below is test code